In [1]:
#Some useful packages

import pandas as pd
import numpy as np
import matplotlib.pylab as plt

In [2]:
#Load data and data reduction and data cleaning

df = pd.read_csv('liquors.csv')
df.drop(['Store Location','County', 'Invoice/Item Number', 'Item Description', 'Item Number'], axis=1, inplace=True)
df.drop(df.index[:10708015], inplace=True)
df.dropna(inplace=True)
df.head()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Date,Store Number,Store Name,Address,City,Zip Code,County Number,Category,Category Name,Vendor Number,Vendor Name,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
10708015,01/03/2017,4312,I-80 Liquor / Council Bluffs,2411 S 24TH ST #1,Council Bluffs,51501,78.0,1012200.0,Scotch Whiskies,55.0,SAZERAC NORTH AMERICA,6,1750,$10.52,$15.78,3,$15.78,5.25,1.38
10708016,01/03/2017,4312,I-80 Liquor / Council Bluffs,2411 S 24TH ST #1,Council Bluffs,51501,78.0,1042100.0,Imported Dry Gins,35.0,BACARDI USA INC,12,750,$10.50,$15.75,1,$189.00,0.75,0.19
10708017,01/03/2017,4312,I-80 Liquor / Council Bluffs,2411 S 24TH ST #1,Council Bluffs,51501,78.0,1082000.0,Imported Cordials & Liqueurs,259.0,Heaven Hill Brands,6,750,$9.83,$14.75,1,$88.50,0.75,0.19
10708018,01/03/2017,4312,I-80 Liquor / Council Bluffs,2411 S 24TH ST #1,Council Bluffs,51501,78.0,1081200.0,Cream Liqueurs,260.0,DIAGEO AMERICAS,12,1000,$16.50,$24.75,2,$297.00,2.00,0.52
10708019,01/03/2017,4312,I-80 Liquor / Council Bluffs,2411 S 24TH ST #1,Council Bluffs,51501,78.0,1012100.0,Canadian Whiskies,65.0,Jim Beam Brands,12,1000,$9.71,$14.57,1,$174.84,1.00,0.26


In [15]:
# Navie Bayes Classifier, part one: Scan data set:

stores = {}
liquors = {}
sls = {}

for i, r in df.iterrows():
    
    store = r['Store Number']
    liquor = r['Category']
    sl = (store, liquor)
    
    if store in stores:
        stores[r['Store Number']] += 1
    elif store not in stores:
        stores[r['Store Number']] = 1
    
    if liquor in liquors:
        liquors[r['Category']] += 1
    elif liquor not in liquors:
        liquors[r['Category']] = 1
        
    if sl in sls:
        sls[sl] += 1
    elif sl not in sls:
        sls[sl] = 1

In [134]:
# Conditional Probabilities :

min_support_stores = 0.005
min_support_liquors = 0.04

# First, we want to find the frequent liqours and the stores that buy liquors frequently with some min_support

frequent_liquors = {}
frequent_stores = {}

S = sum(sls.values())

for k, v in liquors.items():
    if v/S >= min_support_liquors:
        frequent_liquors[k] = v
        
for k, v in stores.items():
    if (v/S) >= min_support_stores:
        frequent_stores[k] = v
        
        
P_s_l = {} # The conditional probabilities that the order comes from the store s given the order is for liquor l
P_l_s = {} # The conditional probabilities that the order is liquor l given that the order comes from the store s

for s, c1 in stores.items():
    for l, c2 in liquors.items():
        jointP = 0
        if (sls.get((s, l))):
            jointP = sls[(s, l)]
        P_s_l[(s, l)] = jointP/c2
        P_l_s[(l, s)] = jointP/c1

In [137]:
distribution_stores = {}

for store in frequent_stores.keys():
    distribution_stores[store] = {}
    for k, v in P_l_s.items():
        l, s = k
        if (s==store):
            distribution_stores[store][l] = v

In [142]:
distribution_liquors = {}

for liquor in frequent_liquors.keys():
    distribution_liquors[liquor] = {}
    for k, v in P_s_l.items():
        s, l = k
        if (l==liquor):
            distribution_liquors[liquor][s] = v